<a href="https://colab.research.google.com/github/brrikcy/nlp/blob/main/Prediction_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
with open('/content/drive/MyDrive/NLP Datasets/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    data = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


[[1, 1561], [1, 1561, 5], [1, 1561, 5, 129], [1, 1561, 5, 129, 34], [647, 4498], [647, 4498, 4499], [226, 5], [226, 5, 1562], [6, 827], [6, 827, 7], [6, 827, 7, 871], [1, 234], [1, 234, 462], [1, 234, 462, 648], [6, 110], [6, 110, 5], [6, 110, 5, 2072], [1, 678], [1, 678, 1360], [1, 678, 1360, 499], [1, 294], [1, 294, 949], [1, 294, 949, 872], [1, 56], [1, 56, 18], [1, 56, 18, 1], [1, 56, 18, 1, 1014], [1, 56, 18, 1, 1014, 873], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 423], [1, 577, 5, 1, 423, 1361], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 1785], [1, 577, 5, 1, 1785, 763], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 3185], [1, 577, 5, 1, 3185, 649], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 764], [1, 577, 5, 1, 764, 1246], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 2073], [1, 577, 5, 1, 2073, 394], [1, 577], [1, 577, 5], [1, 577, 5, 1], [1, 577, 5, 1, 828], [1, 577, 5, 1, 828, 874], [6, 827], [6, 827, 7], [6, 827, 7, 871], [2

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
print(max_sequence_len)

18


In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]




In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
3010/3010 [==============================] - 77s 25ms/step - loss: 6.2253 - accuracy: 0.0777
Epoch 2/10
3010/3010 [==============================] - 73s 24ms/step - loss: 5.4923 - accuracy: 0.1261
Epoch 3/10
 755/3010 [======>.......................] - ETA: 55s - loss: 5.1310 - accuracy: 0.1449

KeyboardInterrupt: 

In [ ]:
seed_text = "the culprit"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 27ms/step
the culprit is a very serious one
